In [18]:
from __future__ import print_function, division
from builtins import range
import numpy as np
from collections import Counter

In [10]:
class Grid: # Environment
  def __init__(self, width, height, start):
    # i is vertical axis, j is horizontal
    self.width = width
    self.height = height
    self.i = start[0]
    self.j = start[1]
    self.start=start

  def set(self, rewards, actions, obey_prob):
    # rewards should be a dict of: (i, j): r (row, col): reward
    # actions should be a dict of: (i, j): A (row, col): list of possible actions
    self.rewards = rewards
    self.actions = actions
    self.obey_prob = obey_prob

  def non_terminal_states(self):
    return self.actions.keys()

  def terminal_states(self):
    return [x for x in self.rewards.keys() if x not in self.actions.keys()]

  def set_state(self, s):
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    return (self.i, self.j)

  def is_terminal(self, s):
    return s not in self.actions

  def stochastic_move(self, action):
    p = np.random.random()
    if p <= self.obey_prob:
      return action
    if action == 'U' or action == 'D':
      return np.random.choice(['L', 'R'])
    elif action == 'L' or action == 'R':
      return np.random.choice(['U', 'D'])

  def move(self, action):
    actual_action = self.stochastic_move(action)
    if actual_action in self.actions[(self.i, self.j)]:
      if actual_action == 'U':
        self.i -= 1
      elif actual_action == 'D':
        self.i += 1
      elif actual_action == 'R':
        self.j += 1
      elif actual_action == 'L':
        self.j -= 1
    return self.rewards.get((self.i, self.j), 0)

  def step(self,action):
    actual_action=self.stochastic_move(action)
    new_state=[self.i,self.j]
    if actual_action in self.actions[(self.i, self.j)]:
      if actual_action == 'U':
        new_state[0] -= 1
      elif actual_action == 'D':
        new_state[0] += 1
      elif actual_action == 'R':
        new_state[1] += 1
      elif actual_action == 'L':
        new_state[1] -= 1
    new_state=tuple(new_state)
    reward=self.rewards[new_state]
    done=self.is_terminal(new_state)
    return new_state,reward,done

  def check_move(self, action):
    i = self.i
    j = self.j
    # check if legal move first
    if action in self.actions[(self.i, self.j)]:
      if action == 'U':
        i -= 1
      elif action == 'D':
        i += 1
      elif action == 'R':
        j += 1
      elif action == 'L':
        j -= 1
    return (i,j)
    

  def get_transition_probs(self, action):
    # returns a list of (probability, s') transition tuples
    probs = []
    state = self.check_move(action)
    probs.append((self.obey_prob, state))
    disobey_prob = 1 - self.obey_prob
    if not (disobey_prob > 0.0):
      return probs
    if action == 'U' or action == 'D':
      state = self.check_move('L')
      probs.append((disobey_prob / 2, state))
      state = self.check_move('R')
      probs.append((disobey_prob / 2, state))
    elif action == 'L' or action == 'R':
      state = self.check_move('U')
      probs.append((disobey_prob / 2, state))
      state = self.check_move('D')
      probs.append((disobey_prob / 2, state))
    return probs

  def game_over(self):
    # returns true if game is over, else false
    # true if we are in a state where no actions are possible
    return (self.i, self.j) not in self.actions

  def all_states(self):
    # possibly buggy but simple way to get all states
    # either a position that has possible next actions
    # or a position that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())


def standard_grid(obey_prob=1.0, step_cost=None):
  # define a grid that describes the reward for arriving at each state
  # and possible actions at each state
  # the grid looks like this
  # x means you can't go there
  # s means start position
  # number means reward at that state
  # .  .  .  1
  # .  x  . -1
  # s  .  .  .
  # obey_brob (float): the probability of obeying the command
  # step_cost (float): a penalty applied each step to minimize the number of moves (-0.1)
  g = Grid(3, 4, (2, 0))
  rewards = {(0, 3): 1, (1, 3): -1}
  actions = {
    (0, 0): ('D', 'R'),
    (0, 1): ('L', 'R'),
    (0, 2): ('L', 'D', 'R'),
    (1, 0): ('U', 'D'),
    (1, 2): ('U', 'D', 'R'),
    (2, 0): ('U', 'R'),
    (2, 1): ('L', 'R'),
    (2, 2): ('L', 'R', 'U'),
    (2, 3): ('L', 'U'),
  }
  g.set(rewards, actions, obey_prob)
  if step_cost is not None:
    g.rewards.update({
      (0, 0): step_cost,
      (0, 1): step_cost,
      (0, 2): step_cost,
      (1, 0): step_cost,
      (1, 2): step_cost,
      (2, 0): step_cost,
      (2, 1): step_cost,
      (2, 2): step_cost,
      (2, 3): step_cost,
    })
  return g

In [4]:
def big_grid(obey_prob=1.0, step_cost=None):

  g = Grid(8, 6, (7, 0))
  rewards = {(1, 5): 1, (2, 5): -1,(2,3):0.5, (3,5):0.5}
  actions = {
    (0, 0): ('D', 'R'),
    (0, 1): ('L', 'D','R'),
    (0, 2): ('L', 'D', 'R'),
    (0, 3): ('L', 'D', 'R'),
    (0, 4): ('L', 'D', 'R'),
    (0, 5): ('L', 'D'),
    (1, 0): ('U', 'D','R'),
    (1, 1): ('U', 'D', 'R','L'),
    (1, 2): ('U', 'D', 'R','L'),
    (1, 3): ('U', 'D', 'R','L'),
    (1, 4): ('U', 'D', 'R','L'),       
    (2, 0): ('U', 'D','R'),
    (2, 1): ('U','L', 'R'),
    (2, 2): ('U', 'D', 'R','L'),
    (2, 3): ('U', 'D', 'R','L'),
    (2, 4): ('U', 'D', 'R','L'),
    (3, 0): ('U','D'),
    (3, 2): ('U', 'D', 'R'),
    (3, 3): ('U', 'D', 'R','L'),  
    (3, 4): ('U', 'D', 'R','L'),    
    (3, 5): ('U', 'D', 'L') , 
    (4, 0): ('U', 'D','R'),
    (4, 1): ('D', 'R','L'),
    (4, 2): ('U', 'D', 'R','L'),
    (4, 3): ('U', 'D', 'R','L'),
    (4, 4): ('U', 'D', 'R','L'),   
    (4, 5): ('U', 'D', 'L'),   
    (5, 0): ('U', 'D','R'),
    (5, 1): ('U','D', 'R','L'),
    (5, 2): ('U', 'D', 'R','L'),
    (5, 3): ('U', 'D', 'R','L'),
    (5, 4): ('U', 'D', 'R','L'),   
    (5, 5): ('U', 'D', 'L'),  
    (6, 0): ('U', 'D','R'),
    (6, 1): ('U','D', 'R','L'),
    (6, 2): ('U', 'D', 'R','L'),
    (6, 3): ('U', 'D', 'R','L'),
    (6, 4): ('U', 'D', 'R','L'),   
    (6, 5): ('U', 'D', 'L'),       
    (7, 0): ('U', 'R'),
    (7, 1): ('U', 'R','L'),
    (7, 2): ('U', 'R','L'),
    (7, 3): ('U', 'R','L'),
    (7, 4): ('U', 'R','L'),   
    (7, 5): ('U', 'L')        
  }

  g.set(rewards, actions, obey_prob)
  if step_cost is not None:
    g.rewards.update({
      (0, 0): step_cost,
      (0, 1): step_cost,
      (0, 2): step_cost,
      (0, 3): step_cost,
      (0, 4): step_cost,
      (0, 5): step_cost,        
      (1, 0): step_cost,
      (1, 1): step_cost,
      (1, 2): step_cost,
      (1, 3): step_cost,
      (1, 4): step_cost,  
      (2, 0): step_cost,
      (2, 1): step_cost,  
      (2, 2): step_cost,
      (2, 4): step_cost,
      (3, 0): step_cost,
      (3, 2): step_cost,
      (3, 3): step_cost,
      (3, 4): step_cost,  
      (4, 0): step_cost,
      (4, 1): step_cost,  
      (4, 2): step_cost,
      (4, 3): step_cost,
      (4, 4): step_cost,
      (4, 5): step_cost,  
      (5, 0): step_cost,
      (5, 1): step_cost,  
      (5, 2): step_cost,
      (5, 3): step_cost,
      (5, 4): step_cost,
      (5, 5): step_cost,  
      (6, 0): step_cost,
      (6, 1): step_cost,  
      (6, 2): step_cost,
      (6, 3): step_cost,
      (6, 4): step_cost,
      (6, 5): step_cost,
      (7, 0): step_cost,
      (7, 1): step_cost,  
      (7, 2): step_cost,
      (7, 3): step_cost,
      (7, 4): step_cost,
      (7, 5): step_cost,         

    })
  return g

In [5]:
def print_values(V,g):
    for i in range(g.width):
        print('------------------------------------------------------------')
        for j in range(g.height):
            v=V.get((i,j),0)
            if v>=0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print('')

def print_policy(P,g):
    for i in range(g.width):
        print('------------------------------------------------------------')
        for j in range(g.height):
            a=P.get((i,j),' ')
            print("  %s  |" % a, end="")
        print('')
        
def max_dict(d):
    max_key=None
    max_val=float('-inf')
    for k,v in d.itmes():
        if v>max_val:
            max_val=v
            max_key=k
    return max_key, max_val

In [6]:
SMALL_ENOUGH=1e-3
GAMMA=0.9
ALL_POSSIBLE_ACTIONS=('U', 'D', 'L', 'R')

def policy_value(grid,V,s,P):
    # finds the highest value action (max_a) from state s, returns the action and value
    grid.set_state(s)
    transititions=grid.get_transition_probs(P[s])
    expected_v=0
    for (prob,state_prime) in transititions:
        expected_v +=prob*V[state_prime]
    v=grid.rewards[s]+GAMMA*expected_v
    return v

def policy_iteration(grid):
    policy=initialize_random_policy()
    V={}
    states=grid.all_states()
    iterations=0
    for s in states:
        V[s]=grid.rewards[s]
    while True:
        iterations+=1
        biggest_change=0
        for s in grid.non_terminal_states():
            old_v=V[s]
            new_v=policy_value(grid,V,s,policy)
            V[s]=new_v
            biggest_change=max(biggest_change,np.abs(old_v-new_v))
        if biggest_change<SMALL_ENOUGH:
            break    
        for s in grid.non_terminal_states():
            grid.set_state(s)
            max_value=float('-inf')
            for m in grid.actions[s]:
                v=0
                T=grid.get_transition_probs(m)
                for (prob,state_prime) in T:
                    v +=prob*V[state_prime]
                if v>max_value:
                    max_value=v
                    policy[s]=m
    return V,iterations,policy

def initialize_random_policy():
    # policy is a lookup table for state -> action
    # we'll randomly choose an action and update as we learn
    policy={}
    for s in grid.non_terminal_states():
        policy[s]=np.random.choice(ALL_POSSIBLE_ACTIONS)
    return policy

    

In [8]:
    grid=standard_grid(obey_prob=0.8,step_cost=-0.1)
    
    print('rewards:')
    print_values(grid.rewards,grid)
    
    V,iteration,policy=policy_iteration(grid)
        
    print('values:')
    print_values(V,grid)
    print('policy:')
    print_policy(policy,grid)
    print('Iterations: ',iteration)

rewards:
------------------------------------
-0.10|-0.10|-0.10| 1.00|
------------------------------------
-0.10| 0.00|-0.10|-1.00|
------------------------------------
-0.10|-0.10|-0.10|-0.10|
values:
------------------------------------
 0.31| 0.51| 0.72| 1.00|
------------------------------------
 0.15| 0.00| 0.36|-1.00|
------------------------------------
 0.01| 0.01| 0.15|-0.09|
policy:
------------------------------------
  R  |  R  |  R  |     |
------------------------------------
  U  |     |  U  |     |
------------------------------------
  U  |  R  |  U  |  L  |
Iterations:  11


In [31]:
    grid=big_grid(obey_prob=0.8,step_cost=-0.03)
    
    print('rewards:')
    print_values(grid.rewards,grid)
    
    V,iteration,policy=policy_iteration(grid)
    
    print('values:')
    print_values(V,grid)
    print('policy:')
    print_policy(policy,grid)
    print('Iterations: ',iteration)

rewards:
------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03| 1.00|
------------------------------------
-0.03|-0.03|-0.03| 0.50|-0.03|-1.00|
------------------------------------
-0.03| 0.00|-0.03|-0.03|-0.03| 0.50|
------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
values:
------------------------------------
 0.97| 1.13| 1.30| 1.48| 1.30| 1.10|
------------------------------------
 1.11| 1.31| 1.53| 1.78| 1.53| 1.00|
------------------------------------
 1.25| 1.50| 1.78| 2.13| 1.81|-1.00|
------------------------------------
 1.06| 0.00| 1.55| 1.81| 1.90| 2.24|
------------------------------------
 0.94| 1.11| 1.33| 1.54| 1.65| 1.91|
---------------------

In [11]:
def huge_grid(obey_prob=1.0, step_cost=None):

  g = Grid(10, 10, (9, 0))
  rewards = {(1, 9): 1, (2, 9): -1,(4,2):-0.2, (5,5):0.2}
  actions = {
      (0,0): ('R','D'),
      (0,9): ('L','D'),
      (9,0): ('R','U'),
      (9,9): ('L','U'),
  }
  for j in range(1,9):
      actions[(0,j)]=('L','R','D')
      actions[(9,j)]=('L','R','U')
  for i in range(1,9):
      actions[(i,0)]=('R','U','D')
  for i in range(3,9):
      actions[(i,9)]=('L','U','D')
  for i in range(1,9):
    for j in range(1,9):
        actions[(i,j)]=('L','R','U','D')
  actions.update({
      (6,4): ('L','U','D'),
      (5,5): ('L','R','U'),
      (6,6): ('R','U','D'),
      (7,5): ('L','R','D'),
      (7,1): ('L','U','D'),
      (6,2): ('L','R','U'),
      (7,3): ('R','U','D'),
      (8,2): (('L','R','D'))       
  })
  del actions[(6,5)]
  del actions[(7,2)]
  
  g.set(rewards, actions, obey_prob)
  if step_cost is not None:
        for i in range(0,10):
            for j in range(0,10):
                if (i,j) not in g.rewards.keys():
                    g.rewards.update({
                        (i,j): step_cost
                    })
  return g

In [12]:
    grid=huge_grid(obey_prob=0.8,step_cost=-0.03)
    
    print('rewards:')
    print_values(grid.rewards,grid)
    print('')
    
    V,iteration,policy=policy_iteration(grid)
    
    print('Values (Policy Iteration Results):')
    print_values(V,grid)
    print('')
    print('Policy (Policy Iteration Results):')
    print_policy(policy,grid)
    print('')
    print('Iterations: ',iteration)

rewards:
------------------------------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03| 1.00|
------------------------------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-1.00|
------------------------------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------------------------------
-0.03|-0.03|-0.20|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03| 0.20|-0.03|-0.03|-0.03|-0.03|
------------------------------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
------------------------------------------------------------
-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|-0.03|
---------------

In [13]:
sum(V.values())/100

0.28945749962376244

In [14]:
num_episodes=100
max_steps=1000
rewards=[]
final_states=[]
for episode in range(num_episodes):
    grid.set_state(grid.start)
    state=(grid.i,grid.j)
    done=False
    rewards_current_episode=0
    for step in range(max_steps):
        action=policy[state]
        new_state,reward,done=grid.step(action)    
        state=new_state
        grid.set_state(state)
        rewards_current_episode+=reward
        if done ==True:
            break
    rewards.append(rewards_current_episode)
    final_states.append(state)

In [17]:
sum(rewards)/100

49.48129999999996

In [19]:
Counter(final_states)

Counter({(1, 9): 47,
         (5, 5): 29,
         (5, 6): 15,
         (6, 6): 2,
         (4, 5): 4,
         (4, 6): 1,
         (4, 4): 1,
         (3, 6): 1})